In [92]:
import pickle

In [93]:
extracted = []
with open('./6893/match_info41k_Nov20.pkl', 'rb') as file:
    extracted = pickle.load(file)

##### Run them separately 

In [ ]:
with open('./6893/FLEXSR_matche_info_save14k_Nov22.pkl', 'rb') as file:
    extracted = pickle.load(file)

In [94]:
len(extracted)

2

# Generate Column Names

In [95]:
import csv
import pandas as pd

In [96]:
column_names = ['gameId', 'winner', 'firstBlood', 'firstTower', 'firstInhibitor', 'firstBaron', 'firstDragon', 'firstRiftHerald', 't1_champ1Name', 't1_champ2Name', 't1_champ3Name', 't1_champ4Name', 't1_champ5Name', 't1_towerKills', 't1_inhibitorKills', 't1_baronKills', 't1_dragonKills', 't1_riftHeraldKills', 't1_totalGold', 't1_totalLevel', 't1_avgLevel', 't1_totalMinionKills', 't1_totalJungleMinionKills', 't1_totalKills', 't1_totalDeath', 't1_totalAssist', 't2_champ1Name', 't2_champ2Name', 't2_champ3Name', 't2_champ4Name', 't2_champ5Name', 't2_towerKills', 't2_inhibitorKills', 't2_baronKills', 't2_dragonKills', 't2_riftHeraldKills', 't2_totalGold', 't2_totalLevel', 't2_avgLevel', 't2_totalMinionKills', 't2_totalJungleMinionKills', 't2_totalKills', 't2_totalDeath', 't2_totalAssist']

In [97]:
print(len(column_names))

61


In [98]:
df = pd.DataFrame(columns=column_names)
df

,gameId,creationTime,gameDuration,seasonId,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,...,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5


In [99]:
def get_first_blood_team(info):
    for participant in match['info']['participants']:
        if participant["firstBloodKill"]:
            # Find the first blood then return the team number.
            # The return value matches the teams' order in info['teams']
            if participant['teamId'] == info['teams'][0]['teamId']:
                return 1
            else:
                return 2
    # No first blood in current match.
    return 0

In [100]:
def get_first_tower_team(info):
    for participant in match['info']['participants']:
        if participant["firstTowerKill"]:
            # Find the first tower then return the team number.
            # The return value matches the teams' order in info['teams']
            if participant['teamId'] == info['teams'][0]['teamId']:
                return 1
            else:
                return 2
    # No first blood in current match.
    return 0

In [101]:
def get_first_inhibitor_team(info):
    # 1 = team1, 2 = team2, 0 = none
    if info['teams'][0]['objectives']['inhibitor']['first']:
        return 1
    elif info['teams'][1]['objectives']['inhibitor']['first']:
        return 2
    else:
        return 0

In [102]:
def get_first_baron_team(info):
    # 1 = team1, 2 = team2, 0 = none
    if info['teams'][0]['objectives']['baron']['first']:
        return 1
    elif info['teams'][1]['objectives']['baron']['first']:
        return 2
    else:
        return 0

In [103]:
def get_first_dragon_team(info):
    # 1 = team1, 2 = team2, 0 = none
    if info['teams'][0]['objectives']['dragon']['first']:
        return 1
    elif info['teams'][1]['objectives']['dragon']['first']:
        return 2
    else:
        return 0

In [104]:
def get_first_rift_herald_team(info):
    # 1 = team1, 2 = team2, 0 = none
    if info['teams'][0]['objectives']['riftHerald']['first']:
        return 1
    elif info['teams'][1]['objectives']['riftHerald']['first']:
        return 2
    else:
        return 0

In [90]:
new_rows = []

for match in extracted:
    
    if match['info']['gameDuration'] < 15 * 60:
        continue 

    # Find the winner for current match
    if match['info']['teams'][0]['win']:
        winner = 1
    else:
        winner = 2
        
    new_row = {
        'gameId': match['info']['gameId'],
        'winner': winner,
        'firstBlood': get_first_blood_team(match['info']),
        'firstTower': get_first_tower_team(match['info']),
        'firstInhibitor': get_first_inhibitor_team(match['info']),
        'firstBaron': get_first_baron_team(match['info']),
        'firstDragon': get_first_dragon_team(match['info']),
        'firstRiftHerald': get_first_rift_herald_team(match['info']),
    }
    
    # Initialize team-wise aggregated stats
    team_stats = {
        't1': {'totalGold': 0, 'totalLevel': 0, 'totalMinionKills': 0, 'totalJungleMinionKills': 0, 'totalDeath': 0, 'totalAssist': 0},
        't2': {'totalGold': 0, 'totalLevel': 0, 'totalMinionKills': 0, 'totalJungleMinionKills': 0, 'totalDeath': 0, 'totalAssist': 0}
    }
    
    # Create champion columns -- 30 columns in total
    team_participant_count = {match['info']['teams'][0]['teamId']: 0, match['info']['teams'][1]['teamId']: 0}
    for participant in match['info']['participants']:
        team_id = 't1' if participant['teamId'] == match['info']['teams'][0]['teamId'] else 't2'
        
        team_participant_count[participant['teamId']] += 1
        player_num = team_participant_count[participant['teamId']]
        
        team_stats[team_id]['totalGold'] += participant['goldEarned']
        team_stats[team_id]['totalLevel'] += participant['champLevel']
        team_stats[team_id]['totalMinionKills'] += participant['totalMinionsKilled']
        team_stats[team_id]['totalJungleMinionKills'] += participant['neutralMinionsKilled']
        team_stats[team_id]['totalDeath'] += participant['deaths']
        team_stats[team_id]['totalAssist'] += participant['assists']
        
        # Champion and summoner spells
        new_row[f'{team_id}_champ{player_num}Name'] = participant['championName']

    # Create objectives columns(5 columns) and bans columns(5 columns) for two teams -- 10 columns for each team
    for index, team in enumerate(match['info']['teams']):
        team_id = f't{index + 1}'
        
        # objective columns
        new_row[f'{team_id}_towerKills'] = team['objectives']['tower']['kills']
        new_row[f'{team_id}_inhibitorKills'] = team['objectives']['inhibitor']['kills']
        new_row[f'{team_id}_baronKills'] = team['objectives']['baron']['kills']
        new_row[f'{team_id}_dragonKills'] = team['objectives']['dragon']['kills']
        new_row[f'{team_id}_riftHeraldKills'] = team['objectives']['riftHerald']['kills']
        new_row[f'{team_id}_totalKills'] = team['objectives']['champion']['kills']
        new_row[f'{team_id}_totalGold'] = team_stats[team_id]['totalGold']
        new_row[f'{team_id}_totalLevel'] = team_stats[team_id]['totalLevel']
        new_row[f'{team_id}_avgLevel'] = team_stats[team_id]['totalLevel'] / 5  # Assuming 5 players per team
        new_row[f'{team_id}_totalMinionKills'] = team_stats[team_id]['totalMinionKills']
        new_row[f'{team_id}_totalJungleMinionKills'] = team_stats[team_id]['totalJungleMinionKills']
        new_row[f'{team_id}_totalDeath'] = team_stats[team_id]['totalDeath']
        new_row[f'{team_id}_totalAssist'] = team_stats[team_id]['totalAssist']

    new_rows.append(new_row)

,gameId,creationTime,gameDuration,seasonId,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,...,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5
0,4828636118,1699843165613,1812,13,1,1,1,1,1,2,...,0,0,0,1,0,85,119,897,203,238
1,4828575524,1699840199497,1515,13,2,2,2,2,2,2,...,8,1,1,2,2,59,122,-1,142,119


In [ ]:
df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
df

In [ ]:
pd.set_option('display.max_columns', None)
df.head()

In [ ]:
len(df)

In [ ]:
df.drop_duplicates(subset='gameId', keep='first', inplace=True)

In [ ]:
df.to_csv('./match_data_final.csv', index=False)